In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime
import warnings
from colorama import Fore, Style
import gc
import pickle

from sklearn.calibration import CalibrationDisplay
from utils import get_data, amex_metric, make_categorical_features

In [3]:
data = pd.read_feather('data/train_data.ftr')

In [4]:
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [5]:
data.groupby('customer_ID')[cat_features].nunique()

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68
customer_ID,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1,1,1,1,1,1,1,1,1,0,1
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,1,1,1,1,1,2,1,1,1,0,1
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,1,1,2,1,1,1,1,1,1,0,1
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,1,1,1,1,2,1,1,1,1,0,3
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,1,2,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,1,1,1,1,2,2,1,1,1,1,1
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,2,2,1,1,2,1,1,1,1,0,1
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,1,2,1,1,1,1,1,1,1,0,1


In [3]:
cat_df = make_categorical_features(data)
if 'target' in data.columns:
    main_data = data.drop('target', axis=1).groupby('customer_ID').agg(['mean', 'median', 'std', 'min', 'max']).sort_index()
else:
    main_data = data.groupby('customer_ID').agg(['mean', 'median', 'std', 'min', 'max']).sort_index()

/var/folders/bb/jzxxldrn52l0stzqmszjx6840000gn/T/ipykernel_17691/3749480365.py:3: FutureWarning: ['S_2', 'D_63', 'D_64', 'D_66', 'D_68', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  main_data = data.drop('target', axis=1).groupby('customer_ID').agg(['mean', 'median', 'std', 'min', 'max']).sort_index()


In [ ]:
def make_categorical_features(df):
    cat_df = pd.DataFrame([], index=df['customer_ID'].unique())
    for feature in cat_features:
        cl_sv = pd.pivot_table(df, index='customer_ID', columns=feature, values = 'P_2', aggfunc = 'count')
        cl_sv['summs'] = cl_sv.sum(axis=1)
        for i in cl_sv.columns[:-1]:
            cl_sv[i] = cl_sv[i] / cl_sv['summs']
        cl_sv.columns = [feature + '_'+ str(i) + '_count' for i in cl_sv.columns]
        cat_df = cat_df.merge(cl_sv, how='left', left_index=True, right_index=True)
    cat_df = cat_df.sort_index().fillna(0)
    del cl_sv
    return cat_df

In [4]:
cat_df

,B_30_0.0_count,B_30_1.0_count,B_30_2.0_count,B_30_summs_count,B_38_1.0_count,B_38_2.0_count,B_38_3.0_count,B_38_4.0_count,B_38_5.0_count,B_38_6.0_count,...,D_66_1.0_count,D_66_summs_count,D_68_0.0_count,D_68_1.0_count,D_68_2.0_count,D_68_3.0_count,D_68_4.0_count,D_68_5.0_count,D_68_6.0_count,D_68_summs_count
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,1.000000,0.000000,0.0,13,0.000000,1.000000,0.000000,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,13
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,1.000000,0.000000,0.0,13,0.000000,1.000000,0.000000,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,13
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,1.000000,0.000000,0.0,13,1.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,13
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,1.000000,0.000000,0.0,13,0.000000,1.000000,0.000000,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.153846,0.692308,0.000000,0.000000,0.153846,13
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,1.000000,0.000000,0.0,13,0.538462,0.461538,0.000000,0.0,0.0,0.0,...,1.0,13,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,1.000000,0.000000,0.0,13,0.000000,1.000000,0.000000,0.0,0.0,0.0,...,1.0,13,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,13
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0.230769,0.769231,0.0,13,0.000000,0.000000,0.076923,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,13
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,1.000000,0.000000,0.0,13,0.000000,0.461538,0.538462,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,13
ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf388145b2c3d01967fcce461,1.000000,0.000000,0.0,13,0.000000,0.000000,1.000000,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.000000,0.307692,0.692308,0.000000,0.000000,13


In [4]:
main_data.columns = [f'{x[0]}_{x[1]}' for x in main_data.columns]
cid = pd.Categorical(data.pop('customer_ID'), ordered=True)
last = (cid != np.roll(cid, -1))
if train: target = data.loc[last, 'target']
df_avg = data[features_avg].groupby(cid).mean().rename(columns={f: f"{f}_avg" for f in features_avg})
data = data.loc[last, features_last].rename(columns={f: f"{f}_last" for f in features_last}).set_index(np.asarray(cid[last]))
data = pd.concat([data, df_avg, cat_df, main_data], axis=1)
cat_features = data.select_dtypes(['category']).columns.tolist()
data[cat_features] = data[cat_features].astype('str')

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,...,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,...,NaN,NaN,0.003956,0.003166,0.005032,NaN,0.009575,0.005493,0.009216,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954102,0.091492,0.021652,1.009766,0.006817,0.123962,0.007599,0.009422,...,NaN,NaN,0.003269,0.007328,0.000427,NaN,0.003429,0.006985,0.002604,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960449,0.002455,0.013687,1.002930,0.001372,0.117188,0.000685,0.005531,...,NaN,NaN,0.006119,0.004517,0.003201,NaN,0.008423,0.006527,0.009598,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947266,0.002483,0.015190,1.000977,0.007607,0.117310,0.004654,0.009308,...,NaN,NaN,0.003672,0.004944,0.008888,NaN,0.001670,0.008125,0.009827,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-11-05,0.979492,0.416016,0.020813,0.828125,0.003487,0.090759,0.005341,0.025146,...,NaN,NaN,0.006836,0.003679,0.000457,NaN,0.000906,0.001497,0.002775,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-12-23,0.984863,0.296631,0.007210,0.812500,0.005905,0.079895,0.002243,0.023697,...,NaN,NaN,0.003309,0.007095,0.007858,NaN,0.002777,0.008224,0.008858,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-01-06,0.982910,0.444092,0.013153,0.815430,0.003456,0.100525,0.002111,0.012344,...,NaN,NaN,0.009956,0.009995,0.001088,NaN,0.005692,0.006775,0.005566,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-02-06,0.969727,0.442627,0.009857,1.003906,0.005116,0.101807,0.009933,0.008575,...,NaN,NaN,0.005543,0.006565,0.009880,NaN,0.008125,0.001168,0.003983,0
